In [1]:
import stanfordnlp

In [2]:
#stanfordnlp.download('ru') 

In [3]:
nlp = stanfordnlp.Pipeline(lang='ru')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\Alexander\\stanfordnlp_resources\\ru_syntagrus_models\\ru_syntagrus_tokenizer.pt', 'lang': 'ru', 'shorthand': 'ru_syntagrus', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\Alexander\\stanfordnlp_resources\\ru_syntagrus_models\\ru_syntagrus_tagger.pt', 'pretrain_path': 'C:\\Users\\Alexander\\stanfordnlp_resources\\ru_syntagrus_models\\ru_syntagrus.pretrain.pt', 'lang': 'ru', 'shorthand': 'ru_syntagrus', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'C:\\Users\\Alexander\\stanfordnlp_resources\\ru_syntagrus_models\\ru_syntagrus_lemmatizer.pt', 'lang': 'ru', 'shorthand': 'ru_syntagrus', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': 'C:\\Users\\Alexander\\stanfo

In [4]:
doc = nlp("Переведи маме сто рублей.")

In [5]:
doc.sentences[0].print_dependencies()

('Переведи', '0', 'root')
('маме', '1', 'iobj')
('сто', '4', 'nummod:gov')
('рублей', '1', 'obl')
('.', '1', 'punct')
